In [2]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

with open('data/reddit-comments-2015-08.csv', 'rb' ) as f:
    reader = csv.reader(f, skipinitialspace = True)
    reader.next()
    # split doc into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(doc[0].decode('utf-8').lower()) for doc in reader])
    sentences = ["%s %s %s" % (sentence_start_token, sent, sentence_end_token) for sent in sentences]
    print "Parsed {} sentences.".format(len(sentences))

Parsed 79170 sentences.


In [31]:
token_sentences = [nltk.word_tokenize(sent) for sent in sentences]
token_freq = nltk.FreqDist(itertools.chain(*token_sentences))
print "Total of {} unique words tokens".format(len(token_freq))

Total of 65751 unique words tokens


In [42]:
vocab = token_freq.most_common(vocabulary_size - 1)
index2vocab = [word[0] for word in vocab]
index2vocab.append(unknown_token)
word2index = {word: i for i, word in enumerate(index2vocab)}
print "With a vocabulary sized restricted to {}".format(len(word2index))
print "The least frequent word is '{}' with {} counts in the reddit comments".format(vocab[-1][0],vocab[-1][1])

With a vocabulary sized restricted to 8000
The least frequent word is 'devoted' with 10 counts in the reddit comments


In [46]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(token_sentences):
    token_sentences[i] = [word if word in word2index else unknown_token for word in sent]

print "\nExample sentence: '{}'".format(sentences[4])
print "\nExample sentence after Pre-processing: '{}'".format(token_sentences[4])


Example sentence: 'SENTENCE_START i put in the rules at a ranking site and noticed that top qbs had 300 points more than the top rb/wr. SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'i', u'put', u'in', u'the', u'rules', u'at', u'a', u'ranking', u'site', u'and', u'noticed', u'that', u'top', u'qbs', u'had', u'300', u'points', u'more', u'than', u'the', u'top', 'UNKNOWN_TOKEN', u'.', u'SENTENCE_END']'


In [47]:
# Create the training data
X_train = np.asarray([[word2index[word] for word in sent[:-1]] for sent in token_sentences])
y_train = np.asarray([[word2index[word] for word in sent[1:]] for sent in token_sentences])

In [50]:
X_train[0:2]

array([ [0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2],
       [0, 11, 17, 7, 3114, 6036, 7999, 7999, 6036, 2]], dtype=object)

In [73]:
print "x:\n{} \n{}".format(" ".join([index2vocab[index] for index in X_train[0]]), X_train[0])
print "y:\n{} \n{}".format(" ".join([index2vocab[index] for index in y_train[0]]), y_train[0])

x:
SENTENCE_START i joined a new league this year and they have different scoring rules than i 'm used to . 
[0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2]
y:
i joined a new league this year and they have different scoring rules than i 'm used to . SENTENCE_END 
[6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2, 1]


In [69]:
class RNN:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        
    def softmax(self, x):
        xt = np.exp(x - np.max(x))
        return xt / np.sum(xt)
        
    def forward_propagation(self, x):
        # total number of time steps
        T = len(x)
        s = np.zeros((T+1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        o = np.zeros((T, self.word_dim))
        for t in np.arange(T):
            s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
            o[t] = self.softmax(self.V.dot(s[t]))     
        return [o, s]
    
    def predict(self, x):
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)

In [70]:
np.random.seed(10)
RNN_model = RNN(vocabulary_size)
o, s = RNN_model.forward_propagation(X_train[10])
print o.shape
print o

(45, 8000)
[[ 0.00012408  0.0001244   0.00012603 ...,  0.00012515  0.00012488
   0.00012508]
 [ 0.00012536  0.00012582  0.00012436 ...,  0.00012482  0.00012456
   0.00012451]
 [ 0.00012387  0.0001252   0.00012474 ...,  0.00012559  0.00012588
   0.00012551]
 ..., 
 [ 0.00012414  0.00012455  0.0001252  ...,  0.00012487  0.00012494
   0.0001263 ]
 [ 0.0001252   0.00012393  0.00012509 ...,  0.00012407  0.00012578
   0.00012502]
 [ 0.00012472  0.0001253   0.00012487 ...,  0.00012463  0.00012536
   0.00012665]]


In [74]:
pred = RNN_model.predict(X_train[10])
print pred.shape
print pred
print " ".join([index2vocab[index] for index in pred])

(45,)
[1284 5221 7653 7430 1013 3562 7366 4860 2212 6601 7299 4556 2481  238 2539
   21 6548  261 1780 2005 1810 5376 4146  477 7051 4832 4991  897 3485   21
 7291 2007 6006  760 4864 2182 6569 2800 2752 6821 4437 7021 7875 6912 3575]
students shortly museum ruining background hunt madden wr chicken immoral hadith lighter rude questions achieve but sells making fill arguing purchase grows feat head lube winners downside states steal but researchers christian utilize fire domain resolution 10-15 genuinely magical worship в branches memes node preferred
